In [ ]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import pickle
from collections import defaultdict
import numpy as np
import scipy
import os
import time       
import os 

from emb.lsa import LSA
from utils.find_ranking_citation import CitationRanking
from utils.write_result import WriteResult
from utils.utils import sort_this
from utils.configuraiton import Rec_configuration 
from utils.eval import Metrics
from utils.data_loading import DataLoading

In [ ]:
class Master:
    def __init__(self, rerank = False):
        self.rec_conf = Rec_configuration()
        self.article_title, self.article_abstract, self.geo_title, \
        self.geo_summary, self.citation_data = DataLoading().get_all_details() #we should get actual citations
        self.auto_rank = CitationRanking(self.citation_data)
        self.write_res = WriteResult()
        choose = list(self.citation_data.keys())
        self.geo_test_data = [key for key in choose if self.citation_data[key] != []] #76,064
        #self.rr = ReRankingTitle() # i definitely need to work on this as well (re-ranking title using bert)
        self.top_threshold = 10 #sounds nice
        self.rerank = rerank
        #bert related
        #self.model_distilbert = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens') #roberta 
        self.model_addr = self.rec_conf.model_path_lsa #subject to change
        self.res_addr = self.rec_conf.result_address_lsa #subject to change

    def process(self):
        #check paths and check vecs  
        lsa = LSA(self.rec_conf.model_path_lsa)
        pmids, lsa_vecs = None, None #this should be loading something

        # This is for loading the pre-trained bert models.
        if lsa_vecs is not None and pmids is not None:
            print('load trained models and encoded publications vectors')
            '''
            
            '''
        else:
            print('Building models')
            joined_dict_article = {}
            for pmid in self.article_title: #for articles, encoding the articles
                #print('pmid')
                #print(pmid)
                if (self.article_title[pmid] + self.article_abstract[pmid]).strip() != '':
                    joined_dict_article[pmid] = self.article_title[pmid] + ' ' + self.article_abstract[pmid]
            #should be some training here, otherwise just do the easiest encodinh
            lsa_dict = lsa.training_LSA(joined_dict_article)
            with open(self.res_addr + 'smallbase/lsa_dict.pickle', 'wb') as handle:
                pickle.dump(lsa_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
   
    
        joined_dict_geo = {} # maxium: 76,064
        # screening first for those geo_ids with pmids 
        for geo_id in self.geo_test_data:
            print('geo_id')
            print(geo_id)
            if (self.geo_title[geo_id] + self.geo_summary[geo_id]).strip() != '':
                #only get the ones with actual texts
                joined_dict_geo[geo_id] = self.geo_title[geo_id] + ' ' + self.geo_summary[geo_id]
        geo_lsa_vecs = lsa.test_LSA(joined_dict_geo)
        with open(self.res_addr + 'smallbase/geo_lsa_vecs.pickle', 'wb') as handle:
            pickle.dump(geo_lsa_vecs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def main():
    x = Master(rerank = False)
    x.process()


if __name__ == '__main__':
    main()

In [ ]:
#read-in stored vec 
res_addr = 'results/lsa_plain/smallbase/'
with open(res_addr +'lsa_dict.pickle', 'rb') as fp:
    lsa_dict = pickle.load(fp)
with open(res_addr +'geo_lsa_vecs.pickle', 'rb') as fp:
      geo_lsa_vecs = pickle.load(fp)        

In [ ]:
#do numpy only
_, _, geo_title, geo_summary, citation_data = DataLoading().get_all_details() #we should get actual citations
auto_rank = CitationRanking(citation_data)
write_res = WriteResult()  
b_size = 64

In [ ]:
chunks = len(geo_lsa_vecs) // b_size +1
for step in range(chunks):
    batch = geo_lsa_vecs[step*b_size:(step+1)*b_size]
    #print(batch)
    #print(batch)
    #batch = batch[0].to(device)
    batch = np.array(batch)
    print(batch.shape)
    geo_ids_batch = geo_ids[step*b_size: (step+1)*b_size]
    #geo_batch, geo_ids_batch = batch
    similarity_value_dict = dict()
    similarity_scores = cosine_similarity(batch, lsa_vecs)
    np.save(res_addr + 'similarity_scores_batch_' + str(step), similarity_scores)#.cpu().numpy())
    #load this later
    #np.load(self.res_addr + 'base/similarity_scores')
    print(similarity_scores.shape)
    #call the re-ranking or not 
    for i, geo_id in enumerate(geo_ids_batch): #this is only from testing geo_id
        print(geo_id)
        similarity_value_dict[geo_id] =  list(similarity_scores[i])#.cpu().numpy())

    new_similarity_dict =  sort_this(geo_sim_dict = similarity_value_dict, pmid_ls = pmids)
    new_similarity_dict = dict(new_similarity_dict)
    filename =  res_addr + 'new_similarity_dict_batch_' + str(step)
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(res_addr + 'new_similarity_dict_batch_' + str(step), 'wb') as handle:
        pickle.dump(new_similarity_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    print('export dict!!!!!!!!')
     
        #screen for geo_ids that actually has citations t
    for id, geo_id in enumerate(geo_ids_batch): 
        print(geo_id)
        #this has already stored in a pickled list of geo_haspmid.pickle
        temp_selected = new_similarity_dict[geo_id]
        filename =  res_addr + geo_id + '.txt'
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        write_res.write(res_addr  + geo_id + '.txt', temp_selected)
        auto_rank.find_citations(geo_id, list(temp_selected.keys())) #keys are a list of pmids

a, b, c, d = auto_rank.get_values() #need to modify this 
print('good geo recommendations = {}, top1 hit geo recommendations = {}, bad geo recommendations = {}, '
      'geo without citations = {}'.format(a, b, c, d))

In [ ]:
#import glob
#files = glob.glob(res_addr + "similarity_scores_batch_*")
res_addr = 'results/lsa_plain/smallbase/'
new_similarity_dict =  {}
#print
#for file in files:
for i in range(step +1):
    with open(res_addr + "new_similarity_dict_batch_" + str(i), 'rb') as fp:
        dict_batch = pickle.load(fp)
        new_similarity_dict.update(dict_batch)
#dict.update(dict2)
#dict.update(dict3)
with open(res_addr + 'new_similarity_dict.pickle', 'wb') as handle:
    pickle.dump(new_similarity_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#MRR 
print('MRR:')
print(Metrics(citation_data).calculate_mrr(new_similarity_dict)) #mrr
#recall @1 and recall @10
print('recall@1, recall@10:')
print(Metrics(citation_data).calculate_recall_at_k(new_similarity_dict, 1))
print(Metrics(citation_data).calculate_recall_at_k(new_similarity_dict, 10))
#Precision@1 and precision10
print('precision@1, precision@10:')
print(Metrics(citation_data).calculate_precision_at_k(new_similarity_dict, 1))        
print(Metrics(citation_data).calculate_precision_at_k(new_similarity_dict, 10))
#MAP@10
print(Metrics(citation_data).calculate_MAP_at_k(new_similarity_dict))

## 2. time it 

In [ ]:
joined_dict_geo = {}
for geo_id in geo_title:
    #print('geo_id')
    #print(geo_id)
    if (geo_title[geo_id] + geo_summary[geo_id]).strip() != '':
        #only get the ones with actual texts
        joined_dict_geo[geo_id] = geo_title[geo_id] + ' ' + geo_summary[geo_id]

In [ ]:
lsa = LSA(Rec_configuration().model_path_lsa)

In [ ]:
lsa_vecs = np.array(list(lsa_dict.values()))
timeit_ls =[]
chunks = len(joined_dict_geo) // b_size +1
for step in range(chunks):
    out = dict(list(joined_dict_geo.items())[step*b_size:(step+1)*b_size])  
    geo_ids_batch = geo_ids[step*b_size: (step+1)*b_size]
    start_time = time.time()
    geo_lsa_vecs = lsa.test_LSA(joined_dict_geo)
    #batch = geo_tfidf_vecs[step*b_size:(step+1)*b_size]
    batch = np.array(list(geo_lsa_vecs.values()))
    #print(batch.shape)
    #geo_ids_batch = geo_ids[step*b_size: (step+1)*b_size]
    similarity_value_dict = dict()
    similarity_scores = cosine_similarity(batch, lsa_vecs)
    #np.save(res_addr + 'similarity_scores_batch_' + str(step), similarity_scores)#.cpu().numpy())
    #load this later
    #np.load(self.res_addr + 'base/similarity_scores')
    #print(similarity_scores.shape)
    for i, geo_id in enumerate(geo_ids_batch): #this is only from testing geo_id
        #print(geo_id)
        similarity_value_dict[geo_id] =  list(similarity_scores[i])#.cpu().numpy())

    new_similarity_dict =  sort_this(geo_sim_dict = similarity_value_dict, pmid_ls = pmids)
    end_time = time.time()
    secs = end_time - start_time
    timeit_ls.append(secs)
    if step > 4:
        break
    
print(np.mean(np.array(timeit_ls))) 
print(np.mean(np.array(timeit_ls))/b_size)